# Advanced Data Features

This notebook demonstrates advanced data capabilities of SurrealEngine:

1. **Materialized Views** - Pre-computed aggregation views for high-performance analytics
2. **Pydantic Integration** - Using SurrealEngine types with Pydantic for validation

In [1]:
pip install pydantic


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Setup
import asyncio
from surrealengine import (
    Document, StringField, FloatField, DateTimeField,
    create_connection, RecordID
)
from surrealengine.materialized_view import Sum, Count, Mean
from datetime import datetime
from typing import List
from pydantic import BaseModel

In [2]:
# Connect to SurrealDB
conn = create_connection(
    url="ws://db:8000/rpc",
    username="root",
    password="root",
    namespace="demo",
    database="advanced_features",
    make_default=True
)
await conn.connect()
print("Connected to SurrealDB!")

Connected to SurrealDB!


## 1. Materialized Views

Materialized views allowing you to pre-compute complex aggregations and query them like regular tables. This is ideal for real-time analytics where running the aggregation on raw data every time would be too slow.

In [3]:
# Define a source document
class SalesOrder(Document):
    product = StringField()
    category = StringField()
    amount = FloatField()
    region = StringField()
    created_at = DateTimeField()
    
    class Meta:
        collection = "sales"

await SalesOrder.create_table()

In [5]:
# Seed some data
categories = ["Electronics", "Clothing", "Home"]
regions = ["North", "South", "East", "West"]
import random

print("Seeding data...")
for i in range(50):
    await SalesOrder(
        product=f"Product {i}",
        category=random.choice(categories),
        amount=random.uniform(10.0, 500.0),
        region=random.choice(regions),
        created_at=datetime.now()
    ).save()
print("Seeded 50 orders.")

Seeding data...
Seeded 50 orders.


In [4]:
# Define a Materialized View using the aggregation builder
# View: Sales by Category and Region

view_query = SalesOrder.objects.aggregate() \
    .group(
        by_fields=["category", "region"],
        total_sales=Sum("amount"),
        avg_sale=Mean("amount"),
        order_count=Count()
    )

# Create the view in the database
# It will automatically refresh continuously (LIVE) or simpler depending on DB config. 
# Note: SurrealDB currently has no explicit syntax for 'MATERIALIZED VIEW', 
# but we simulate it via pre-computed table definitions if supported, 
# or this API is a wrapper around DEFINE TABLE ... AS SELECT ...

SalesSummary = SalesOrder.create_materialized_view(
    name="sales_summary",
    query=view_query
)
await SalesSummary.create()

print("Materialized View 'sales_summary' created.")

Materialized View 'sales_summary' created.


In [5]:
# Query the Materialized View like a normal table
# This is instant, as data is pre-computed

summary = await SalesSummary.objects.order_by("total_sales", "DESC").all()

print("--- Sales Summary (Pre-computed) ---")
for row in summary:
    # Row is a dictionary-like object (or typed if we defined a wrapper)
    print(f"{row['category']} - {row['region']}: ${row['total_sales']:.2f} ({row['order_count']} orders)")

--- Sales Summary (Pre-computed) ---
Clothing - South: $2344.59 (8 orders)
Electronics - North: $2085.97 (7 orders)
Clothing - East: $1700.91 (7 orders)
Home - South: $1677.23 (6 orders)
Home - West: $1598.40 (5 orders)
Home - East: $1500.41 (4 orders)
Electronics - East: $1339.41 (5 orders)
Electronics - South: $905.59 (2 orders)
Electronics - West: $593.15 (3 orders)
Clothing - West: $581.91 (2 orders)
Home - North: $412.24 (1 orders)


## 2. Pydantic Integration

SurrealEngine's `RecordID` and other types are compatible with Pydantic v2, allowing you to use them directly in your API schemas.

In [6]:
# Define a Pydantic model
class UserResponse(BaseModel):
    id: RecordID
    username: str
    email: str
    active: bool = True

# Simulate data from DB (RecordID object)
db_record = {
    "id": RecordID("users", "user123"),
    "username": "jdoe",
    "email": "john@example.com"
}

# Validate using Pydantic
user = UserResponse(**db_record)
print(f"Validated User: {user}")
print(f"User Type: {type(user.id)}")

# Serialize to JSON (RecordID serializes to string "table:id")
json_output = user.model_dump_json()
print(f"JSON Output: {json_output}")

Validated User: id=RecordID(table_name=users, record_id='user123') username='jdoe' email='john@example.com' active=True
User Type: <class 'surrealdb.data.types.record_id.RecordID'>
JSON Output: {"id":"users:user123","username":"jdoe","email":"john@example.com","active":true}


In [7]:
# Parse from JSON string (e.g. from API request)
json_input = '{"id": "users:user123", "username": "jdoe", "email": "john@example.com"}'

parsed_user = UserResponse.model_validate_json(json_input)
print(f"Parsed ID: {parsed_user.id} (Table: {parsed_user.id.table_name}, ID: {parsed_user.id.id})")

Parsed ID: users:user123 (Table: users, ID: user123)


## Cleanup

In [ ]:
await conn.disconnect()
print("Disconnected.")